# Encontrando los Límites Originales de Oraciones Preprocesadas

Dado un texto inicial que puede provenir de un conversor de PDF a TXT con múltiples errores.
Este se procesa y las oraciones son convertidas a nuevas oraciones despojadas de estos errores.
¿Cómo retornar las posiciones originales de estas oraciones y su contenido original una vez 
obtenidas los límites exactos tras el procesamiento?

In [1]:
import sys
import os
from preprocess.normalize import add_doc_ending_point
import swalign

PATH = '/home/abelma/preprocess/test/'



Abrimos el fichero para hacer las pruebas.

In [2]:
text = open(os.path.join(PATH,'test_text.txt')).read()
print (text[:round(len(text)/4)])

For other (optional) flags of <opencv_createsamples>, see the official... documentation
at http://Docs.opencv.org/doc/user_guide/ug_traincascade.html.
[ 99 ]
www.it-ebooks.info.
Generating Haar Cascades for Custom 8.4 Targets
Creating <cascade> by running:
<opencv_traincascade>
3. anoche.
4 Después. . . 

Over 110 recipes to master this full-stack Python web 
framework
1.	 Take


__el mismo texto analizado manualmente:__

Mostremos por un instante el mismo texto con un análisis hecho a mano de las oraciones.

In [3]:
texth = open(os.path.join(PATH,'test_text_human_analysis.txt')).read()
print (text[:round(len(text)/4)])

For other (optional) flags of <opencv_createsamples>, see the official... documentation
at http://Docs.opencv.org/doc/user_guide/ug_traincascade.html.
[ 99 ]
www.it-ebooks.info.
Generating Haar Cascades for Custom 8.4 Targets
Creating <cascade> by running:
<opencv_traincascade>
3. anoche.
4 Después. . . 

Over 110 recipes to master this full-stack Python web 
framework
1.	 Take


# Primera Función

La primera función creada para estos fines por mí, fue para el PAN 2014, donde desafortunadamente no pude participar por
no haber Wifi o políticas de SSH en la UCLV que me permitieran contectarme a la PC virtual del evento. Veamos!

In [4]:
def find_sentences(text,orig_text):
    """ Tokeniz a given text and return a dict containing all start and 
    end positions for each sentence.

    :param text: generated text from clean_punctuation function.
    :param type: string
    :param orig_text: original text in process.
    :param type: string
    
    :returns sentence_list: contain the sentence' number, the 
    generated sentence(only letters, '_' and the end-points), the 
    init-position of the original sentence equivalent to the "generated
    sentence", and the end-position of the original sentence equivalent to the "generated sentence".
    :rtype: tuple list.
    
    .. author: Abel Meneses abad
    Finish on Fri, 28 Feb 2014
    Next_revision on Sun Aug 3 2014
    """
    sentence_list = []
    sentence_count = 0
    init, end = 0, 0

    for i in range(len(text)):
        if text[i] == '.':  #if '.' is found 
            end = i         #end sentence = '.' pos
            while text[end] not in LETTERS: # in spite of found the last letter sentence pos
                end-=1 
            sentence = text[init:end+1]     # then construct exactly the sentence without end '.'
            tuple = (sentence_count,sentence,init,end+1)
            sentence_list.append(tuple)
            sentence_count += 1
            init = i
            restante = text[init:]

        elif init < len(text) and text[init] not in LETTERS:
            init+=1
            restante = text[init:]

    count = 0
    words = []
    new_origen = 0
    len_quitadas = 0
    orig_text2 = orig_text
    sentence_list2 = []
    for i in range(len(sentence_list)):
        oracion_buscada = sentence_list[i][1]
        for word in oracion_buscada.split():
            words.append(word)

        lengt_oracion_buscada = len(oracion_buscada)
        new_origen = orig_text.find(words[0])
        orig_pos_init = new_origen + len_quitadas   #Dónde aparece la primera palabra de la oración que estoy buscando.
        orig_pos_end = orig_pos_init + lengt_oracion_buscada # Posición inicial + largo de la oración = posición final.

        tuple = (i, oracion_buscada, orig_pos_init, orig_pos_end) #construye la tupla con los valores del doc original.
        sentence_list2.append(tuple)
        restante = orig_text[new_origen+lengt_oracion_buscada:] # Elimino del texto la oración encontrada, queda de primera la siguiente

        orig_text = orig_text[new_origen+lengt_oracion_buscada:]    # Elimino del texto la oración encontrada, queda de primera la siguiente
        len_quitadas = orig_pos_end
        words = []

    sentence_list = sentence_list2

    return sentence_list

# Probando

In [5]:
import string
LETTERS = ''.join([string.ascii_letters,string.digits,'ñÑáéíóúÁÉÍÓÚüÜ'])
textr = open(os.path.join(PATH,'test_text_result.txt')).read()

In [6]:
#Comprobando
sentenceList = find_sentences(textr,text)
print (sentenceList[0][1])
print (text[sentenceList[0][2]:sentenceList[0][3]])

For other optional flags of opencv_createsamples see the official documentation at http___Docs_opencv_org_doc_user_guide_ug_traincascade_html
For other (optional) flags of <opencv_createsamples>, see the official... documentation
at http://Docs.opencv.org/doc/user_guide/ug_traincasc


# Resultado

Como ven suele fallar, así que vamos a probar una versión utilizando el algoritmo de Smith-Waterman de alineamiento de textos. También usado en las medidas de similitud de cadenas.

In [7]:
#For swalign algthm
# choose your own values here… 2 and -1 are common.
match = 2
mismatch = -1
scoring = swalign.NucleotideScoringMatrix(match, mismatch)

sw = swalign.LocalAlignment(scoring)  # you can also choose gap penalties, etc...
alignment = sw.align('ACACACTA','AGCACACA')
alignment.dump()

Query: 1 AGCACAC-A 8
         | ||||| |
Ref  : 1 A-CACACTA 8

Score: 12
Matches: 7 (77.8%)
Mismatches: 2
CIGAR: 1M1I5M1D1M


**Probemos un ejemplo real con nuestros textos.**

In [8]:
textr = open('/home/abelm/preprocess/test/test_text_result.txt').read()
s1 = textr[:143]
text = open('/home/abelm/preprocess/test/test_text.txt').read()
s2 = text[:]
s2 = s2.replace('\n',' ')
alignment = sw.align(s1,s2)
alignment.dump()

Query:   1 For other (optional) flags of <opencv_createsamples>, see the official... documentation at http://Docs.opencv.org/doc/user_guide/ug_traincascade.html.  151
           |||||||||| |||||||| |||||||||| ||||||||||||||||||||  |||||||||||||||||   ||||||||||||||||||||||...||||.||||||.|||.|||.||||||||||.|||||||||||||||.|||| |
Ref  :   1 For other -optional- flags of -opencv_createsamples-- see the official--- documentation at http___Docs_opencv_org_doc_user_guide_ug_traincascade_html-  142

Score: 248
Matches: 133 (88.1%)
Mismatches: 18
CIGAR: 10M1I8M1I10M1I20M2I17M3I76M1I1M


Como se puede ver utilizando los puntos que existen en la primera oración (71, 72, 73, 103, 110, 145 y 150) la similitud con
la oración preprocesada va desde 92% al 88%.
Si utilizamos el texto completo la función encuentra la porción donde hay mayor coincidencia. En el ejemplo mostrado se nota que el **Query** desde 1 hasta 151 es la oración buscada.

Note al correr esta celda que para textos grandes esto podría demorar.
Utilicemos un ardid surgido de la experimentación: buscar el punto inicial(que es el 0 o el último punto de la oración anterior
encontrada), y utilicemos el tamaño de **s1** para comparar el fragmento final de s1 con cualquiera del mismo tamaño 
desde donde quiera que encuentro un punto en **s2** (71, 72, 73, 103, 110, 145 y 150), hacia atrás.

In [9]:
def cut(string,num, w):
    """Función auxiliar para cortar un texto dada un intervalo [w,num]
    Solo sirve para el próximo bloque. No se usa en el algoritmo final
    aunque sí se usó la idea de cortar para reducir los tiempos de cómputo.
    """
    stri = string[:num]
    stri = stri.replace('\n',' ')
    strig = stri[-round(w/4):]
    return strig

Nótese que en el bloque siguiente los puntos finales (71, 72,...) fueron calculados manualmente para poder analizar que ocurría.
Este sería el procedimiento normal durante una investigación.
En el algoritmo final estas posiciones se calculan automáticamente al detectar cada posición de un punto.

In [10]:
textr = open('/home/abelm/preprocess/test/test_text_result.txt').read()
sr = textr[:143]
w = len(sr)
s1 = sr[-round(w/4):]
text = open('/home/abelm/preprocess/test/test_text.txt').read()
s2 = cut(text,71,w)
alignment = sw.align(s1,s2)
alignment.dump();print(71)
s2 = cut(text,72,w)
alignment = sw.align(s1,s2)
alignment.dump();print(72)
s2 = cut(text,73,w)
alignment = sw.align(s1,s2)
alignment.dump();print(73)
s2 = cut(text,103,w)
alignment = sw.align(s1,s2)
alignment.dump();print(103)
s2 = cut(text,110,w)
alignment = sw.align(s1,s2)
alignment.dump();print(110)
s2 = cut(text,145,w)
alignment = sw.align(s1,s2)
alignment.dump();print(145)
print (s1,'\n',s2,'\n****************')
s2 = cut(text,150,w)
alignment = sw.align(s1,s2)
alignment.dump();print(150)
print (s1,'\n',s2,'\n****************')
s2 = cut(text,162,w)
alignment = sw.align(s1,s2)
alignment.dump();print(162)
print (s1,'\n',s2,'\n****************')
s2 = cut(text,172,w)
alignment = sw.align(s1,s2)
alignment.dump();print(172)

Query: 31 i-cia 34
          | | |
Ref  : 21 inc-a 24

Score: 4
Matches: 3 (60.0%)
Mismatches: 2
CIGAR: 1M1D1M1I1M
71
Query: 30 i-cia 33
          | | |
Ref  : 21 inc-a 24

Score: 4
Matches: 3 (60.0%)
Mismatches: 2
CIGAR: 1M1D1M1I1M
72
Query: 29 i-cia 32
          | | |
Ref  : 21 inc-a 24

Score: 4
Matches: 3 (60.0%)
Mismatches: 2
CIGAR: 1M1D1M1I1M
73
Query: 15 t-ation a 22
          | | | |.|
Ref  : 18 tra-i-nca 24

Score: 6
Matches: 5 (55.6%)
Mismatches: 4
CIGAR: 1M1D1M1I1M1I3M
103
Query:  8 t-ation a 15
          | | | |.|
Ref  : 18 tra-i-nca 24

Score: 6
Matches: 5 (55.6%)
Mismatches: 4
CIGAR: 1M1D1M1I1M1I3M
110
Query:  7 oc/user_guide/ug_traincascade 35
          ||.||||||||||.|||||||||||||||
Ref  :  1 oc_user_guide_ug_traincascade 29

Score: 52
Matches: 27 (93.1%)
Mismatches: 2
CIGAR: 29M
145
oc_user_guide_ug_traincascade_html . 
 .org/doc/user_guide/ug_traincascade. 
****************
Query:  2 oc/user_guide/ug_traincascade.html-. 36
          ||.||||||||||.|||||||||||||||.|||| |

In [11]:
textr = open('/home/abelm/preprocess/test/test_text_result.txt').read()
sr = textr[144:167]
w = len(sr)
s1 = sr[-round(w/4):]
text2 = open('/home/abelm/preprocess/test/test_text.txt').read()
text = text2[151:217]
s2 = cut(text,11,w)
alignment = sw.align(s1,s2)
alignment.dump();print(11)
print (s1,'\n',s2,'\n****************')
s2 = cut(text,21,w)
alignment = sw.align(s1,s2)
alignment.dump();print(21)
print (s1,'\n',s2,'\n****************')
s2 = cut(text,26,w)
alignment = sw.align(s1,s2)
alignment.dump();print(26)
print (s1,'\n',s2,'\n****************')
s2 = cut(text,65,w)
alignment = sw.align(s1,s2)
alignment.dump();print(65)
print (s1,'\n',s2,'\n****************')

Query: 6 . 6
         |
Ref  : 6 . 6

Score: 2
Matches: 1 (100.0%)
Mismatches: 0
CIGAR: 1M
11
info . 
 ] www. 
****************
Query: 6 . 6
         |
Ref  : 6 . 6

Score: 2
Matches: 1 (100.0%)
Mismatches: 0
CIGAR: 1M
21
info . 
 books. 
****************
Query: 2 info-. 6
         |||| |
Ref  : 1 info . 6

Score: 9
Matches: 5 (83.3%)
Mismatches: 1
CIGAR: 4M1D1M
26
info . 
 .info. 
****************
Query: 2 om 8. 6
         | | |
Ref  : 4 o- -. 6

Score: 4
Matches: 3 (60.0%)
Mismatches: 2
CIGAR: 1M1I1M1I1M
65
info . 
 tom 8. 
****************


## Conclusiones parciales

Como se puede observar en ambos ejemplos el mayor score lo da en la posición donde coincide el punto final de la oración
de la que fue editada.
Y el experimento de medir la distancia desde un cuarto de la oración hasta el final ofrece mayor velocidad.
Implementemos ahora el algoritmo final, implementado en el [2.2 Aligning Preprocessed Sentence to Originals](02.2c-Jaccard-Align-Preproc-to-Original-Sent.ipynb)